### 00 라이브리러 임포트

In [8]:
import bt
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta

from data.get_data import get_prices_df, get_pdf_df, get_base_price_df
from data.fetch_data import fetch_index_info

from core.get_backtest import get_base_backtest, get_eql_backtest, get_mkw_backtest, get_bdd_mkw_backtest
from core.get_weigh import get_cap_weigh, get_bdd_cap_weigh


### 01 설정
- ticker에 맞게 확인해볼 수 있습니다
- 그래프 컬러도 설정합니다

In [9]:
class CONFIG:
    ticker = "AIQ"
    
    upper_bound=fetch_index_info(etf_tkr=ticker)[0]['upper_bound']
    
    start_date = datetime.now() - relativedelta(years=1)
    
    template="simple_white"
    primary_color="#F58220"
    secondary_color="#CDCECB"

In [10]:
def get_miraeasset_theme_figure(data, title):
    fig = px.line(data, 
              template=CONFIG.template,
              title=title,
              labels={
                  "value": "YTD",
                  "index": "date",
                  "variable": "ETFs"
              })

    fig['data'][0]['line']['color']=CONFIG.primary_color
    fig['data'][1]['line']['color']=CONFIG.secondary_color
    
    return fig

### 02 CONIFG의 ticker 가격과 구성종목들의 가격 로드
- 미리 만들어둔 데이터베이스에서 로드합니다

In [11]:
etf_price = get_base_price_df(etf_tkr=CONFIG.ticker, start_date=CONFIG.start_date).dropna()

pdf_df = get_pdf_df(etf_tkr=CONFIG.ticker)

tkrs = pdf_df.child_stk_tkr.to_list()

child_prices = get_prices_df(tickers=tkrs, start_date=CONFIG.start_date)

### 03 동일가중
- 리밸런싱: 3개월

In [12]:
base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
eql_backtest = get_eql_backtest(name=f"{CONFIG.ticker} 동일가중", child_prices=child_prices)

ret = bt.run(eql_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 동일가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).rebase(),
                                  title=TITLE)

fig.show()

### 04 시가총액가중(유동시가총액)
- 리밸런싱: 3개월

In [13]:
quartely_child_cap_weigh = get_cap_weigh(child_prices=child_prices, pdf_df=pdf_df)

base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
mkw_backtest = get_mkw_backtest(name=f"{CONFIG.ticker} 시총가중",
                                child_prices=child_prices,
                                weigh=quartely_child_cap_weigh)

ret = bt.run(mkw_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 시총가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).loc[quartely_child_cap_weigh.index[0]:].rebase(),
                                  title=TITLE)

fig.show()

### 05 최대제한 시가총액가중(유동비율)
- 리밸런싱 3개월
- "ETF방식을그대로" 옵션에 해당합니다.
- 최소비중 및 리밸런싱 주기, 현금비중은 고려하지 않아 다소 차이가 있을 수 있습니다.

In [14]:
quartely_child_cap_upper_bound_weigh = get_bdd_cap_weigh(child_prices=child_prices,
                                                         pdf_df=pdf_df,
                                                         upper_bound=CONFIG.upper_bound)

base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
bdd_mkw_backtest = get_bdd_mkw_backtest(name=f"{CONFIG.ticker} 최대제한 시총가중",
                                child_prices=child_prices,
                                weigh=quartely_child_cap_upper_bound_weigh)

ret = bt.run(bdd_mkw_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 최대제한 시총가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].rebase(),
                                  title=TITLE)

fig.show()